In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [ ]:
%load_ext tensorboard

# Clear all tensorboards from previous runs.
!rm -rf ./runs/*

In [ ]:
# Exercise 1.

batch_size = 64
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),])

# import data for training purposes.
full_train_set = torchvision.datasets.SVHN(root='./root', split='train', download=True, transform=transform)
# split data training, 80% training, 20% validation.
train_size = int(0.8 * len(full_train_set))
validation_size = len(full_train_set) - train_size
train_set, validation_set = torch.utils.data.random_split(full_train_set, [train_size, validation_size])
# craets dataloader for train data.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=2)

# import and loads data for testing purposes.
test_set = torchvision.datasets.SVHN(root='./root', split='test', download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [ ]:
# Exercise 1 - checking that the data has been loaded correctly

# choose from train_loader, validation_loader, test_loader
data_set_checker = train_loader

data_iter = iter(data_set_checker)
images, labels = data_iter.next()

# text style
BOLD = '\033[1m'
END = '\033[0m'

print(BOLD+"information about Batch size and images sizes: "+END, images.shape)
print(BOLD+"size of label array: "+END,labels.shape)
print(BOLD+"labels for example: "+END, labels)
# random_img = np.random.seed(data_set_checker)
# print("random image from data set: ")
# plt.imshow(random_img)

In [ ]:
i = 40
# print(images[i].shape)
# print(images[i].numpy())
img = images[i].permute(1,2,0).numpy()
# print(img.shape)
# print(img)
plt.imshow(img)
print("the label is: ", labels[i])

In [ ]:
figure = plt.figure()
num_of_images = 60

for i in range(1, num_of_images+1):
  plt.subplot(6, 10, i)
  plt.axis('off')
  plt.imshow(images[i].squeeze().permute(1,2,0), cmap=None)

In [ ]:
# Exercise 2 - Fully Connected NN

input_size = 32 * 32 * 3
output_size = 10

class FC_Net(nn.Module):
    def __init__(self, hidden_layer_size_1, hidden_layer_size_2):
      super(FC_Net, self).__init__()
      self.L1 = nn.Linear(input_size, hidden_layer_size_1)
      self.L2 = nn.Linear(hidden_layer_size_1, hidden_layer_size_2)
      self.L3 = nn.Linear(hidden_layer_size_2, output_size)

    def forward(self, x):
      # Flatten out the image before processing.
      x = x.view(x.shape[0], -1)
      x = self.L1(x)
      x = F.relu(x)
      x = self.L2(x)
      x = F.relu(x)
      x = self.L3(x)
      
      return x

In [ ]:
# Exercise 2 - Convolutional NN.

class Conv_Net(nn.Module):
    def __init__(self, kernel_size):
      super(Conv_Net, self).__init__()
      self.L1 = nn.Conv2d(3, 10, kernel_size, padding=1, stride=1)
      self.L2 = nn.MaxPool2d(2, stride=2, padding=0)
      self.L3 = nn.Conv2d(10, 20, kernel_size, padding=1, stride=1)
      self.L4 = nn.MaxPool2d(2, stride=2, padding=0)
      self.L5 = nn.Linear(20*((32//4)**2), L2)
      self.L6 = nn.Linear(L2, L3)

    def forward(self, x):
      x = self.L1(x)
      x = F.relu(x)
      x = self.L2(x)
      x = self.L3(x)
      x = F.relu(x)
      x = self.L4(x)
      x = x.view(x.shape[0], -1)
      x = self.L5(x)
      x = F.relu(x)
      x = self.L6(x)

      return x

In [ ]:
# Exercise 3 - the train function.

epochs = 30
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()

def train(net, optimizer, criterion, epochs, title):
  loss_board = SummaryWriter(f'runs/{title}_losses')
  acc_board = SummaryWriter(f'runs/{title}_accs')
  for epoch in range(epochs):
    print(f'epoch: {epoch+1}/{epochs}\n{"-"*10}')
    for phase in ["train", "validate"]:
      # You can replace "trainloader" and "validloader" with whatever you name the loaders in exercise 1.
      loader = trainloader if phase == "train" else validloader

      for inputs, labels in loader:
        # Implement the actual training here. Make sure that when phase == "validation" the network doesn't actually learn.
        # I took care of logging accuracy so you see how tensorboard works. You still need to log losses in loss_board though.

        # Logging accuracy on each epoch.
        # This line uses a field "outputs" which doesn't exist yet. This is supposed to be what you name the output of the forward pass.
        accuracy = (torch.Tensor(list(map(lambda w: torch.argmax(w), outputs))) == labels).float().sum() / batch_size
        acc_board.add_scalar(f'{title}_{phase}_acc', accuracy, epoch)
  loss_board.flush()
  loss_board.close()
  acc_board.flush()
  acc_board.close()

In [ ]:
# Exercise 4 - FC Net with hidden layer sizes 512 and 256.

fc_512_256 = FC_Net(512, 256)
fc_512_256_optimizer = optim.Adam(fc_512_256.parameters(), lr=learning_rate)
train(fcnn, fc_512_256_optimizer, criterion, epochs, "FNN_512_256")

%tensorboard --logdir=runs/FNN_512_256_losses
%tensorboard --logdir=runs/FNN_512_256_accs

In [ ]:
# Exercise 4 - FC Net with hidden layer sizes 64 and 32.

fc_64_32 = FC_Net(64, 32)
fc_64_32_optimizer = optim.Adam(fc_64_32.parameters(), lr=learning_rate)
train(fcnn, fc_64_32_optimizer, criterion, epochs, "FNN_64_32")

%tensorboard --logdir=runs/FNN_64_32_losses
%tensorboard --logdir=runs/FNN_64_32_accs

In [ ]:
# Exercise 4 - CNN with kernel size of 10.

cnn_10 = Conv_Net(10)
cnn_10_optimizer = optim.Adam(cnn_10.parameters(), lr=learning_rate)
train(fcnn, cnn_10_optimizer, criterion, epochs, "CN_10")

%tensorboard --logdir=runs/CNN_10_losses
%tensorboard --logdir=runs/CNN_10_accs

In [ ]:
# Exercise 4 - CNN with kernel size of 1.

cnn_1 = Conv_Net(1)
cnn_1_optimizer = optim.Adam(cnn_1.parameters(), lr=learning_rate)
train(fcnn, cnn_1_optimizer, criterion, epochs, "CN_1")

%tensorboard --logdir=runs/CNN_1_losses
%tensorboard --logdir=runs/CNN_1_accs